## Data Augmentation

After hyperparameter tuning, the model accuracy has increased drastically. Athough there still seems room for improvement. We can try data augmentation for better generalization and increasing the accuracy.

here I will be using the `preprocess_image` function with the augment flag set to True to augment our data. We will be rotating and shearing the images.

In [ ]:
# Apply augmentation only to the training set
X_train_augmented = np.empty_like(X_train)
for i in range(len(X_train)):
    X_train_augmented[i] = preprocess_image(Image.fromarray(X_train[i].astype(np.uint8)), augment=True) # augmentation

train_dataset_aug = tf.data.Dataset.from_tensor_slices((X_train_augmented, {"activity_output": y_activity_train, "person_output": y_person_train}))

# shuffle
train_dataset_aug = train_dataset_aug.shuffle(buffer_size=len(X_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Build the model
model_aug = build_model(learning_rate=lr)

# Get the callbacks
callbacks = get_callbacks(lr)

# Train the model
history = model_aug.fit(train_dataset_aug,  # Use the augmented dataset
                    validation_data=val_dataset,
                    batch_size=16, epochs=80,
                    callbacks=callbacks)

Epoch 1/80
180/180 ━━━━━━━━━━━━━━━━━━━━ 92s 196ms/step - activity_output_accuracy: 0.1636 - loss: 29.6372 - person_output_accuracy: 0.6494 - val_activity_output_accuracy: 0.5542 - val_loss: 23.2341 - val_person_output_accuracy: 0.7792 - learning_rate: 1.0000e-04
Epoch 2/80
180/180 ━━━━━━━━━━━━━━━━━━━━ 32s 176ms/step - activity_output_accuracy: 0.6583 - loss: 21.5314 - person_output_accuracy: 0.8048 - val_activity_output_accuracy: 0.6056 - val_loss: 18.5476 - val_person_output_accuracy: 0.7250 - learning_rate: 9.0000e-05
Epoch 3/80
180/180 ━━━━━━━━━━━━━━━━━━━━ 32s 176ms/step - activity_output_accuracy: 0.8381 - loss: 16.5650 - person_output_accuracy: 0.9058 - val_activity_output_accuracy: 0.6444 - val_loss: 15.0122 - val_person_output_accuracy: 0.8167 - learning_rate: 8.1000e-05
Epoch 4/80
180/180 ━━━━━━━━━━━━━━━━━━━━ 32s 178ms/step - activity_output_accuracy: 0.9355 - loss: 13.0395 - person_output_accuracy: 0.9614 - val_activity_output_accuracy: 0.6639 - val_loss: 12.5342 - val_person_

In [ ]:
predictions = model_aug.predict(X_val)

activity_predictions = predictions[0]
person_predictions = predictions[1]


23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 203ms/step


In [ ]:
# Evaluate the model on the validation dataset
eval_results = model_aug.evaluate(val_dataset)

# Print evaluation results
print(f"Validation loss: {eval_results[0]}")
print(f"Validation activity accuracy: {eval_results[1]}")
print(f"Validation person accuracy: {eval_results[2]}")

45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - activity_output_accuracy: 0.7308 - loss: 1.7576 - person_output_accuracy: 0.8468
Validation loss: 1.737059235572815
Validation activity accuracy: 0.7527777552604675
Validation person accuracy: 0.8458333611488342


Observations:


*   The Augmented model shows similar results with a slightly better performance on the `Class` classification accuracy but a reduction in accuracy with the `MoreThanOnePerson` accuracy.
*   The loss here is slightly lesser.
*  We will now try the tuned model with lesser epochs to avoid overfitting.